# Visualize the data 

In [11]:
#coding=utf-8
import json
import geopandas
import matplotlib.pyplot as plt
import requests
import pandas as pd
from shapely.geometry import LineString
import time
import os
import numpy as np
import matplotlib as mpl
import seaborn as sns#添加Seaborn模块
import folium
import webbrowser

In [2]:
 def plot_show(data) :
    data.plot(column='speed', cmap='RdYlGn', legend=True)
    plt.show()

In [3]:
data  =  pd.read_csv("traffic_data/ShangHai_2019_12_23.csv")[['Timestamp','Lcode','speed','CongestionRate']] 
data = data [data["CongestionRate"] >= 1] 
data = data.drop_duplicates(
    keep='first',
    inplace=False)
data = data.reset_index(drop=True)
data

,Timestamp,Lcode,speed,CongestionRate
0,2019_12_22 21:50:00,4465,51,2
1,2019_12_22 21:50:00,4054,100,1
2,2019_12_22 21:50:00,4056,100,1
3,2019_12_22 21:50:00,4061,100,1
4,2019_12_22 21:50:00,4063,100,1
...,...,...,...,...
3127270,2019_12_23 21:48:00,2425,30,1
3127271,2019_12_23 21:48:00,2427,30,1
3127272,2019_12_23 21:48:00,2430,30,1
3127273,2019_12_23 21:48:00,-2427,45,1


In [3]:
import os
filePath = '/Users/jackyanghc/Desktop/Jupyter/'
filename = []
for i,j,k in os.walk(filePath):
    for m in j:
        if m[0] == 'S' and m[13] == "1":
            filename.append(m)

ShangHai = geopandas.read_file(filePath+filename[0],encoding='utf-8')
for i in range(1,len(filename)):
    tmp = geopandas.read_file(filePath+filename[i],encoding='utf-8')
    ShangHai = pd.concat([ShangHai,tmp])

print(ShangHai)

         name            Timestamp  \
0       申嘉湖高速  2019_11_21 16:24:00   
1       申嘉湖高速  2019_11_21 16:24:00   
2      上海绕城高速  2019_11_21 16:24:00   
3      上海绕城高速  2019_11_21 16:24:00   
4       ３２４省道  2019_11_21 16:24:00   
...       ...                  ...   
92128     华申路  2019_11_21 12:00:00   
92129     华申路  2019_11_21 12:00:00   
92130    五洲大道  2019_11_21 12:00:00   
92131     申江路  2019_11_21 12:00:00   
92132     东靖路  2019_11_21 12:00:00   

                                                   Lcode  speed Congestion  \
0      4054,4056,4061,4063,4604,4066,4069,4070,4072,4...     95          1   
1      -4084,-4083,-4079,-4078,-4076,-4072,-4070,-406...    100          1   
2      2011,2010,2009,2008,2007,2006,2005,2392,2001,2...     74          1   
3      -1994,-1995,-1996,-1998,-2000,-2001,-2392,-200...     80          1   
4      4096,4098,4106,4107,4110,4116,4127,4128,4129,4...     45          1   
...                                                  ...    ...        ... 

In [4]:
ShangHai = ShangHai.drop_duplicates(
    keep='first',
    inplace=False)
ShangHai = ShangHai.reset_index(drop=True)

In [5]:
information = ShangHai[['name','Timestamp','Lcode','speed','Congestion']].sort_values(by="Timestamp",ascending= True)  
information = information.reset_index(drop = True)
maps  = {'1':1,'2':2,'3':3,'4':4,'5':5,'0':0}
information.replace(maps,inplace = True)
print(information)

         name            Timestamp  \
0         长宁路  2019_11_21 08:16:00   
1        世纪大道  2019_11_21 08:16:00   
2       内环高架路  2019_11_21 08:16:00   
3       内环高架路  2019_11_21 08:16:00   
4         打浦路  2019_11_21 08:16:00   
...       ...                  ...   
783570    政本路  2019_11_21 21:00:00   
783571   伊敏河路  2019_11_21 21:00:00   
783572    密云路  2019_11_21 21:00:00   
783573    密云路  2019_11_21 21:00:00   
783574    华山路  2019_11_21 21:00:00   

                                             Lcode  speed  Congestion  
0                                      473,472,471     30           1  
1       531,532,533,4636,4991,535,536,537,538,1335     40           1  
2                                               36     50           1  
3                                 -11,-36,-35,-178     60           1  
4                                              501     30           1  
...                                            ...    ...         ...  
783570                           -2255,

# 针对一条道路进行全天的速度和堵塞进行可视化

In [6]:
from pyecharts import options as opts
from pyecharts.charts import Line,Bar,Scatter

In [12]:
def select_on_Lcode(lcode):
    Road_info_time = []
    Road_info_speed = []
    Road_info_congest = []
    Road_info_lcode = [lcode]
    for i in range(0,len(information)):
        if lcode in information.loc[i,'Lcode'].split(","):
            tmp = information.loc[i].values
            Road_info_time.append(tmp[1])
            Road_info_speed.append(int(tmp[3]))
            Road_info_congest.append((int(tmp[4])*5))
    return [Road_info_lcode,Road_info_time,Road_info_speed,Road_info_congest]

In [13]:
def line_base(name_y, x, y, y_c, number = 0,title=None, subtitle=None) -> Line:
    if number != 0:
        y = [i + number for i in y]
        y_c = [i + number for i in y_c]
    l = (
        Line()
        .add_xaxis(x)
        .add_yaxis(name_y[0], y ,is_smooth=True,
            markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average")]),)
        .add_yaxis(name_y[0]+"_congestion", y_c,is_smooth=True,
            markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average")]),)
#         .set_series_opts(
#             areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
#             label_opts=opts.LabelOpts(is_show=False),
#         )
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title=title),
#             xaxis_opts=opts.AxisOpts(
#                 axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
#                 is_scale=False,
#                 boundary_gap=False,
#             ),
#         )
    )
#     s = (
#         Scatter()
#         .add_xaxis(x)
#         .add_yaxis(name_y[0], y_c)
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="Scatter-VisualMap(Color)"),
#             visualmap_opts=opts.VisualMapOpts(max_=150),
#         )
#     )
#     l.overlap(s)
    return l

NameError: name 'Line' is not defined

In [9]:
info = select_on_Lcode('5088')
a  = line_base(info[0], info[1], info[2], info[3],title='Speed vs Time')
info = select_on_Lcode('5086')
b  = line_base(info[0], info[1], info[2], info[3],number =100, title='Speed vs Time')
a.overlap(b)
a.render_notebook()

# 时间段的平均速度，平均拥塞速度

In [10]:
grouped_by_Lcode = information[['Congestion','Timestamp','Lcode']].groupby(['Timestamp','Congestion'],)

In [11]:
grouped_by_Lcode.sum().reset_index()

,Timestamp,Congestion,Lcode
0,2019_11_21 08:16:00,0,"3245-1615,-43621613-51914512,4514,4516,4519-45..."
1,2019_11_21 08:16:00,1,"473,472,471531,532,533,4636,4991,535,536,537,5..."
2,2019_11_21 08:16:00,2,"26352911,2912543,542610-1486-1363,-13622663,26..."
3,2019_11_21 08:16:00,3,"343-315231134649-2671-3455874,58752919,2920-29..."
4,2019_11_21 08:18:00,0,"-49191149-2346,-18453922-75-17083389,2369,2368..."
...,...,...,...
1319,2019_11_21 20:58:00,3,"360-2635,-26343453838-5202,-386811244789,47903..."
1320,2019_11_21 21:00:00,0,"-49193928,3923,3922-4760,-475247606217,6218451..."
1321,2019_11_21 21:00:00,1,"791,792,793,795,4510,796,4981,797,798,799,1129..."
1322,2019_11_21 21:00:00,2,"1663-3160100-2391-3394-2512-2684-5064,-2679-42..."


In [12]:
grouped_by_Congestion = information[['Congestion','Timestamp','speed']].groupby(['Timestamp','Congestion'],)
s = grouped_by_Congestion.count().reset_index()
timestamp = ['2019_11_21 08:16:00','2019_11_21 08:18:00','2019_11_21 08:20:00']
x = ['0 Unknown','1 Unimpeded','2 Amble','3 Congestion','4 Heavy Congestion' ]
all_y = []
for i in timestamp:
    info = s[s['Timestamp'] == i]
    y = list(info['speed'].values)
    while len(y) != 5:
        y.append(0)
    for i in range(5):
        y[i] = int(y[i])
    all_y.append(y)

In [13]:
def bar_base(name_y, x, y, title=None, subtitle=None) -> Bar:
    l = (
        Bar()
        .add_xaxis(x)
        .add_yaxis(name_y[0],y[0])
        .add_yaxis(name_y[1], y[1])
        .add_yaxis(name_y[2], y[2])
        .set_global_opts(title_opts=opts.TitleOpts(title=title))
    )
#     s = (
#         Scatter()
#         .add_xaxis(x)
#         .add_yaxis(name_y[0], y_c)
#         .set_global_opts(
#             title_opts=opts.TitleOpts(title="Scatter-VisualMap(Color)"),
#             visualmap_opts=opts.VisualMapOpts(max_=150),
#         )
#     )
#     l.overlap(s)
    return l

In [14]:
bar_base(timestamp,x,all_y,title='某时刻拥堵值平均统计情况').render_notebook()

# 根据路名来可视化检测

In [14]:
def get_Road_info(lcode,color):
    route = []
    codes = []
    colors = []
    count = 0
    j = 0
    for code in lcode:
        try:
            ro = []
            s = Roads[Roads['Lcode'] == str(code)]
            if len(s) != 0:
                for m in s['geometry'].values:
                    q = list(m.coords)
                    count += len(q)
                for i in q:
                    ro.append([i[1], i[0]])
                j += 1
                route.append(ro)
                colors.append(color[j])
                codes.append(code)
        except:
            print("Not Found", code)
    return route, codes, colors


def line_map(all_data, maps,codes,color):
    colors = ['black','green','yellow','orange','red']
    
    for i in range(0, len(all_data)):
        folium.PolyLine(
            all_data[i],
            weight=4,  # 粗细
            opacity=0.8,  # 透明度
            color=colors[color[i]]).add_to(maps)
#         folium.Marker(
#             all_data[i][0],
#             icon=folium.Icon(color='orange'),
#             popup= codes[i]).add_to(maps)
    
    return maps

# CSV File Load

# 区域收缩到上海市的内环

In [15]:
def is_inBox(x,y):
    A = [121.357288,31.10965]
    B = [121.672458,31.155495]
    C = [121.620959,31.386143]
    D = [121.324329,31.34041]
    a = (B[0] - A[0])*(y - A[1]) - (B[1] - A[1])*(x - A[0])
    b = (C[0] - B[0])*(y - B[1]) - (C[1]- B[1])*(x - B[0])
    c = (D[0] - C[0])*(y - C[1]) - (D[1] - C[1])*(x - C[0])
    d = (A[0] - D[0])*(y - D[1]) - (A[1] - D[1])*(x - D[0])
    if (a > 0 and b > 0 and c > 0 and d > 0) or (a < 0 and b < 0 and c < 0 and d < 0) :
        return 1
    else:
        return 0

In [16]:
def get_lcode_in_area():
    codes = []
    for item in range(0,len(Roads)):
        s = Roads.loc[item]
        q = list(s['geometry'].coords)
        for i in q:
            if is_inBox(i[0],i[1]) == 1:
                codes.append(s['Lcode'])
                break
    return codes

![](https://tva1.sinaimg.cn/large/006y8mN6ly1g9935m1mlnj30re0mdtm2.jpg)
![](https://tva1.sinaimg.cn/large/006y8mN6ly1g993712sabj30vt0kg1kx.jpg)

In [19]:
data = pd.read_csv("../traffic_data/ShangHai_2019_12_23.csv")[['Timestamp','Lcode','speed','CongestionRate']] 
data = data [data["CongestionRate"] >= 1] 
data = data.drop_duplicates(
    keep='first',
    inplace=False)
datas = data.reset_index(drop=True)

Roads = geopandas.read_file("../information_of_road",encoding='utf-8')
print(Roads)

date = pd.to_datetime(datas.Timestamp, format = '%Y_%m_%d %H:%M:%S')
datas.Timestamp = date
datas = datas.set_index('Timestamp')
print(datas)

data_on_hour = datas['2019-12-23 09']

some_data = data_on_hour.groupby(by = 'Lcode')
set_code = some_data.count()
s = set_code [set_code['speed'] >= 15]
code_on_moment = list(s['speed'].index)
codes_on_area = get_lcode_in_area()

code_to_use = []
for i in code_on_moment:
    if str(i) in codes_on_area:
        code_to_use.append(str(i))

      Lcode    lenght   name  \
0     -1434  0.009444    川桥路   
1     -1433  0.018483    川桥路   
2      5892  0.031206   纪白公路   
3      5891  0.044168   纪白公路   
4     -2443  0.035816   沪渝高速   
...     ...       ...    ...   
4301  -6159  0.013064   高科东路   
4302  -1353  0.008597    高青路   
4303  -4667  0.004168   龙华东路   
4304   2074  0.087180   沪昆高速   
4305   1163  0.003516  陆家嘴环路   

                                               geometry  
0     LINESTRING (121.62517 31.24854, 121.62305 31.2...  
1     LINESTRING (121.61598 31.24638, 121.61567 31.2...  
2     LINESTRING (121.18341 31.24684, 121.18413 31.2...  
3     LINESTRING (121.13939 31.24717, 121.13973 31.2...  
4     LINESTRING (121.31937 31.16127, 121.31917 31.1...  
...                                                 ...  
4301  LINESTRING (121.66649 31.21636, 121.66627 31.2...  
4302  LINESTRING (121.53978 31.16507, 121.53921 31.1...  
4303  LINESTRING (121.47839 31.19514, 121.47788 31.1...  
4304  LINESTRING (121.23338 31.0207

In [22]:
len(code_to_use)

3541

In [20]:
print(data_on_hour)

                     Lcode  speed  CongestionRate
Timestamp                                        
2019-12-23 09:00:00   4054    100               1
2019-12-23 09:00:00   4056    100               1
2019-12-23 09:00:00   4061    100               1
2019-12-23 09:00:00   4063    100               1
2019-12-23 09:00:00   4604    100               1
...                    ...    ...             ...
2019-12-23 09:58:00   2425     30               1
2019-12-23 09:58:00   2427     30               1
2019-12-23 09:58:00   2430     30               1
2019-12-23 09:58:00  -2427     40               1
2019-12-23 09:58:00  -2425     40               1

[185489 rows x 3 columns]


In [24]:
lcode = code_to_use
x,y  = 121.473658,31.230378
maps = folium.Map(
    location=[y, x],
    zoom_start=11,
    tiles=
    'http://webrd01.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    attr="高德地图")

set_index = list(set(data_on_hour.index))
for i in range(0,5):
    print(1)
    data_on_miute = data_on_hour[data_on_hour.index == set_index[i]]
    color = list(data_on_miute['CongestionRate'].values)
    all_data,codes,colors = get_Road_info(lcode,color)
    line_map(all_data, maps, codes, colors).save("../roadmap_html/Roadmap{ioc}.html".format(ioc = i))

1
1
1
1
1
